# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Import Libraries</h1></span>

In [15]:
!pip install catboost  polars
import torch
import polars as pl#和pandas类似,但是处理大型数据集有更好的性能.
import pandas as pd#导入csv文件的库
import numpy as np#对矩阵进行科学计算的库
#kfold
from sklearn.model_selection import StratifiedGroupKFold
#model
# from  lightgbm import LGBMRegressor,log_evaluation,early_stopping
from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
import dill#对对象进行序列化和反序列化(例如保存和加载树模型)
from sklearn.feature_extraction.text import TfidfVectorizer#将文本数据转换为tfidf特征
import re#用于正则表达式提取的库
import gc#垃圾回收的库
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Preprocessor</h1></span>

In [38]:
class Preprocessor():
    def __init__(self,seed=2024,target='utility_agent1',train=None,num_folds=10,CV_LB_path="/kaggle/input/mcts-eda-about-cv-and-lb/1016CV_LB.csv"):
        self.seed=seed
        self.target=target
        self.train=train
        self.model_paths=[]#训练和推理的模型
        self.tfidf_paths=[]#字符串的tfidf模型
        self.num_folds=num_folds
        #检查CV和LB情况的CV,LB统计表
#         self.check=pd.read_csv(CV_LB_path)
    
    #Clean the string columns of df
    def clean(self,df,col):
        #String missing value filling
        df[col]=df[col].fillna("nan")
        #Convert string to lowercase
        df[col]=df[col].apply(lambda x:x.lower())
        #Consider this string ‘MCTS-UCB1-0.6-NST-false‘
        ps='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        for p in ps:
            df[col]=df[col].apply(lambda x:x.replace(p,' '))
        return df
    
    #https://www.nhooo.com/note/qa0tpe.html
    #The Automatic Readability Index (ARI) basically tells about difficulty level of text to read
    #1–5: Easy texts (lower elementary grades).
    #6–10: Intermediate texts (upper elementary to middle school).
    #11–14: More difficult texts (high school to early college).
    #15 and above: Very complex texts (college graduate level).
    def ARI(self,txt):
        characters=len(txt)
        words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))#空格,换行符,句号,问号,感叹号,逗号分开.
        sentence=len(re.split('\\.|\\?|\\!',txt))#句号,问号,感叹号分开的句子.
        ari_score=4.71*(characters/words)+0.5*(words/sentence)-21.43
        return ari_score
    """
    http://www.supermagnus.com/mac/Word_Counter/index.html
    McAlpine EFLAW© Test - another way to measure readability 
         (W + SW) / S
    McAlpine EFLAW© Readability
         Scale:
         1-20: Easy
         21-25: Quite Easy
         26-29: Mildly Difficult
         ≥ 30: Very Confusing
         S:total sentences
         W:total words
    """
    
    def McAlpine_EFLAW(self,txt):
        W=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))#空格,换行符,句号,问号,感叹号,逗号分开.
        S=len(re.split('\\.|\\?|\\!',txt))#句号,问号,感叹号分开的句子.
        mcalpine_eflaw_score=(W+S*W)/S
        return mcalpine_eflaw_score
    """
    https://readable.com/readability/coleman-liau-readability-index/
    CLRI: another way to measure readability 
    =0.0588*L-0.296*S-15.8
    L是每100个单词有多少个字母,S是平均每100个单词有多少句子.
    """
    def CLRI(self, txt):
        characters = len(txt)
        words=len(re.split(' |\\n|\\.|\\?|\\!|\,',txt))  # Split by space, newline, period, question mark, exclamation mark, comma.
        sentence=len(re.split('\\.|\\?|\\!',txt))# Sentences separated by periods, question marks, and exclamation marks.
        L=100*characters/words
        S=100*sentence/words
        clri_score=0.0588*L-0.296*S-15.8
        return clri_score
        
        
    # Save the trained tree model, obj is the saved model, path is the path where it needs to be saved
    def pickle_dump(self,obj, path):
        # Open the specified path, binary write
        with open(path, mode="wb") as f:
            # Save the obj object to f, using protocol version 4 for serialization
            dill.dump(obj, f, protocol=4)
    def pickle_load(self, path):
        # Open the specified path, binary read
        with open(path, mode="rb") as f:
            # Load the model according to the specified path
            data = dill.load(f)
            return data
    

    
    # Iterate through all columns of the dataframe df to modify data types and reduce memory usage
    def reduce_mem_usage(self, df, float16_as32=True):
        # memory_usage() is the memory usage of each column in df, sum is to sum them up, B->KB->MB
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

        for col in df.columns:  # Iterate through each column name
            col_type = df[col].dtype  # Column type
            if col_type != object and str(col_type) != 'category':  # Not object, meaning this is a numerical variable
                c_min, c_max = df[col].min(),df[col].max() # Find the maximum and minimum values of this column
                if str(col_type)[:3] == 'int': # If it is an int type variable, regardless of whether it is int8, int16, int32, or int64
                    # If the range of this column's values is within the range of int8, then convert the type (-128 to 127)
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    # If the range of values in this column is within the range of int16, then convert the type (-32,768 to 32,767)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    # If the range of values in this column is within the range of int32, then convert the type (-2,147,483,648 to 2,147,483,647)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    # If the range of values in this column is within the range of int64, then convert the type (-9,223,372,036,854,775,808 to 9,223,372,036,854,775,807)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else: # If it is a floating-point type.
                    # If the value is within the range of float16, if higher precision is needed, consider float32
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        if float16_as32:  # If the data requires higher precision, float32 can be chosen
                            df[col] = df[col].astype(np.float32)
                        else:
                            df[col] = df[col].astype(np.float16)  
                    # If the value is within the range of float32, convert its type
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    # If the value is within the range of float64, convert its type
                    else:
                        df[col] = df[col].astype(np.float64)    
        
        

        #Calculate the memory after completion
        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        # Compared to the initial memory, how much has it decreased?
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        return df
        
    #Feature Engineering
    def FE(self,df,mode='train'): 
        print(f"FE:{mode}")
        df['area']=df['NumRows']*df['NumColumns']
        df['row_equal_col']=(df['NumColumns']==df['NumRows']).astype(np.int8)
        df['Playouts/Moves'] = df['PlayoutsPerSecond'] / (df['MovesPerSecond'] + 1e-15)
        df['EfficiencyPerPlayout'] = df['MovesPerSecond'] / (df['PlayoutsPerSecond'] + 1e-15)
        df['TurnsDurationEfficiency'] = df['DurationActions'] / (df['DurationTurnsStdDev'] + 1e-15)
        df['AdvantageBalanceRatio'] = df['AdvantageP1'] / (df['Balance'] + 1e-15)
        df['ActionTimeEfficiency'] = df['DurationActions'] / (df['MovesPerSecond'] + 1e-15)
        df['StandardizedTurnsEfficiency'] = df['DurationTurnsStdDev'] / (df['DurationActions'] + 1e-15)
        df['AdvantageTimeImpact'] = df['AdvantageP1'] / (df['DurationActions'] + 1e-15)
        df['DurationToComplexityRatio'] = df['DurationActions'] / (df['StateTreeComplexity'] + 1e-15)
        df['NormalizedGameTreeComplexity'] =  df['GameTreeComplexity'] /  (df['StateTreeComplexity'] + 1e-15)
        df['ComplexityBalanceInteraction'] =  df['Balance'] *  df['GameTreeComplexity']
        df['OverallComplexity'] =  df['StateTreeComplexity'] +  df['GameTreeComplexity']
        df['ComplexityPerPlayout'] =  df['GameTreeComplexity'] /  (df['PlayoutsPerSecond'] + 1e-15)
        df['TurnsNotTimeouts/Moves'] = df['DurationTurnsNotTimeouts'] / (df['MovesPerSecond'] + 1e-15)
        df['Timeouts/DurationActions'] = df['Timeouts'] / (df['DurationActions'] + 1e-15)
        df['OutcomeUniformity/AdvantageP1'] = df['OutcomeUniformity'] / (df['AdvantageP1'] + 1e-15)
        df['ComplexDecisionRatio'] = df['StepDecisionToEnemy'] + df['SlideDecisionToEnemy'] + df['HopDecisionMoreThanOne']
        df['AggressiveActionsRatio'] = df['StepDecisionToEnemy'] + df['HopDecisionEnemyToEnemy'] + df['HopDecisionFriendToEnemy'] + df['SlideDecisionToEnemy']
        
        print("deal with outliers")
        df['PlayoutsPerSecond']=df['PlayoutsPerSecond'].clip(0,25000)
        df['MovesPerSecond']=df['MovesPerSecond'].clip(0,1000000)
        
        print("agent1 agent2 feature")
        cols=['selection','exploration_const','playout','score_bounds']
        for i in range(len(cols)):
            for j in range(2):
                df[f'{cols[i]}{j+1}']=df[f'agent{j+1}'].apply(lambda x:x.split('-')[i+1])
        

        print(f"one_hot_encoder")
        #The cols of nunique greater than 2 and less than 10 on the training set
#         onehot_cols=[['NumOffDiagonalDirections', [0.0, 4.82, 2.0, 5.18, 3.08, 0.06]], ['NumLayers', [1, 0, 4, 5]], ['NumPhasesBoard', [3, 2, 1, 5, 4]], ['NumContainers', [1, 4, 3, 2]], ['NumDice', [0, 2, 1, 4, 6, 3, 5, 7]], ['ProposeDecisionFrequency', [0.0, 0.05, 0.01]], ['PromotionDecisionFrequency', [0.0, 0.01, 0.03, 0.02, 0.11, 0.05, 0.04]], ['SlideDecisionToFriendFrequency', [0.0, 0.19, 0.06]], ['LeapDecisionToEnemyFrequency', [0.0, 0.04, 0.01, 0.02, 0.07, 0.03, 0.14, 0.08]], ['HopDecisionFriendToFriendFrequency', [0.0, 0.13, 0.09]], ['HopDecisionEnemyToEnemyFrequency', [0.0, 0.01, 0.2, 0.03]], ['HopDecisionFriendToEnemyFrequency', [0.0, 0.01, 0.09, 0.25, 0.02]], ['FromToDecisionFrequency', [0.0, 0.38, 1.0, 0.31, 0.94, 0.67]], ['ProposeEffectFrequency', [0.0, 0.01, 0.03]], ['PushEffectFrequency', [0.0, 0.5, 0.96, 0.25]], ['FlipFrequency', [0.0, 0.87, 1.0, 0.96]], ['SetCountFrequency', [0.0, 0.62, 0.54, 0.02]], ['DirectionCaptureFrequency', [0.0, 0.55, 0.54]], ['EncloseCaptureFrequency', [0.0, 0.08, 0.1, 0.07, 0.12, 0.02, 0.09]], ['InterveneCaptureFrequency', [0.0, 0.01, 0.14, 0.04]], ['SurroundCaptureFrequency', [0.0, 0.01, 0.03, 0.02]], ['NumPlayPhase', [1, 2, 3, 4, 5, 6, 7, 8]], ['LineLossFrequency', [0.0, 0.96, 0.87, 0.46, 0.26, 0.88, 0.94]], ['ConnectionEndFrequency', [0.0, 0.19, 1.0, 0.23, 0.94, 0.35, 0.97]], ['ConnectionLossFrequency', [0.0, 0.54, 0.78]], ['GroupEndFrequency', [0.0, 1.0, 0.11, 0.79]], ['GroupWinFrequency', [0.0, 0.11, 1.0]], ['LoopEndFrequency', [0.0, 0.14, 0.66]], ['LoopWinFrequency', [0.0, 0.14, 0.66]], ['PatternEndFrequency', [0.0, 0.63, 0.35]], ['PatternWinFrequency', [0.0, 0.63, 0.35]], ['NoTargetPieceWinFrequency', [0.0, 0.72, 0.77, 0.95, 0.32, 1.0]], ['EliminatePiecesLossFrequency', [0.0, 0.85, 0.96, 0.68]], ['EliminatePiecesDrawFrequency', [0.0, 0.03, 0.91, 1.0, 0.36, 0.86]], ['NoOwnPiecesLossFrequency', [0.0, 1.0, 0.68]], ['FillEndFrequency', [0.0, 1.0, 0.04, 0.01, 0.99, 0.72]], ['FillWinFrequency', [0.0, 1.0, 0.04, 0.01, 0.99]], ['ReachDrawFrequency', [0.0, 0.9, 0.98]], ['ScoringLossFrequency', [0.0, 0.6, 0.62]], ['NoMovesLossFrequency', [0.0, 1.0, 0.13, 0.06]], ['NoMovesDrawFrequency', [0.0, 0.01, 0.04, 0.03, 0.22]], ['BoardSitesOccupiedChangeNumTimes', [0.0, 0.06, 0.42, 0.12, 0.14, 0.94]], ['BranchingFactorChangeNumTimesn', [0.0, 0.3, 0.02, 0.07, 0.04, 0.13, 0.01, 0.21, 0.03]], ['PieceNumberChangeNumTimes', [0.0, 0.06, 0.42, 0.12, 0.14, 1.0]], ['selection1', ['ProgressiveHistory', 'UCB1', 'UCB1GRAVE', 'UCB1Tuned']], ['selection2', ['ProgressiveHistory', 'UCB1GRAVE', 'UCB1', 'UCB1Tuned']], ['exploration_const1', ['0.1', '0.6', '1.41421356237']], ['exploration_const2', ['0.6', '0.1', '1.41421356237']], ['playout1', ['MAST', 'NST', 'Random200']], ['playout2', ['Random200', 'NST', 'MAST']]]
        onehot_cols = [['selection1', ['ProgressiveHistory', 'UCB1', 'UCB1GRAVE', 'UCB1Tuned']], ['selection2', ['ProgressiveHistory', 'UCB1GRAVE', 'UCB1', 'UCB1Tuned']], ['exploration_const1', ['0.1', '0.6', '1.41421356237']], ['exploration_const2', ['0.6', '0.1', '1.41421356237']], ['playout1', ['MAST', 'NST', 'Random200']], ['playout2', ['Random200', 'NST', 'MAST']]]
        for col,unique in onehot_cols:
            for u in unique:
                df[f'{col}_{u}']=(df[col]==u).astype(np.int8)
                
     
       
        df['PlayoutsPerSecond/MovesPerSecond']=df['PlayoutsPerSecond']/df['MovesPerSecond']
        
        drop_cols = []
        #The useless features screened out by the model have an occurrence frequency of less than 1%.
        drop_cols= drop_cols + ['Cooperation', 'Team', 'TriangleShape', 'DiamondShape', 'SpiralShape', 'StarShape', 'SquarePyramidalShape', 'SemiRegularTiling', 'CircleTiling', 'SpiralTiling', 'MancalaThreeRows', 'MancalaSixRows', 'MancalaCircular', 'AlquerqueBoardWithOneTriangle', 'AlquerqueBoardWithTwoTriangles', 'AlquerqueBoardWithFourTriangles', 'AlquerqueBoardWithEightTriangles', 'ThreeMensMorrisBoard', 'ThreeMensMorrisBoardWithTwoTriangles', 'NineMensMorrisBoard', 'StarBoard', 'PachisiBoard', 'Boardless', 'NumColumns', 'NumCorners', 'NumOffDiagonalDirections', 'NumLayers', 'NumCentreSites', 'NumConvexCorners', 'NumPhasesBoard', 'NumContainers', 'Piece', 'PieceValue', 'PieceRotation', 'PieceDirection', 'LargePiece', 'Tile', 'NumComponentsType', 'NumDice', 'OpeningContract', 'SwapOption', 'Repetition', 'TurnKo', 'PositionalSuperko', 'AutoMove', 'InitialRandomPlacement', 'InitialScore', 'InitialCost', 'Moves', 'VoteDecision', 'SwapPlayersDecision', 'SwapPlayersDecisionFrequency', 'ProposeDecision', 'ProposeDecisionFrequency', 'PromotionDecisionFrequency', 'RotationDecision', 'RotationDecisionFrequency', 'StepDecisionToFriend', 'StepDecisionToFriendFrequency', 'StepDecisionToEnemy', 'SlideDecisionToEnemy', 'SlideDecisionToEnemyFrequency', 'SlideDecisionToFriend', 'SlideDecisionToFriendFrequency', 'LeapDecision', 'LeapDecisionFrequency', 'LeapDecisionToEmpty', 'LeapDecisionToEmptyFrequency', 'LeapDecisionToEnemy', 'LeapDecisionToEnemyFrequency', 'HopDecisionFriendToEmpty', 'HopDecisionFriendToEmptyFrequency', 'HopDecisionFriendToFriendFrequency', 'HopDecisionEnemyToEnemy', 'HopDecisionEnemyToEnemyFrequency', 'HopDecisionFriendToEnemy', 'HopDecisionFriendToEnemyFrequency', 'FromToDecisionFrequency', 'FromToDecisionEnemy', 'FromToDecisionEnemyFrequency', 'FromToDecisionFriend', 'SwapPiecesDecision', 'SwapPiecesDecisionFrequency', 'ShootDecision', 'ShootDecisionFrequency', 'VoteEffect', 'SwapPlayersEffect', 'PassEffect', 'ProposeEffect', 'ProposeEffectFrequency', 'AddEffectFrequency', 'SowFrequency', 'SowCapture', 'SowCaptureFrequency', 'SowRemove', 'SowBacktracking', 'SowBacktrackingFrequency', 'SowProperties', 'SowOriginFirst', 'SowCCW', 'PromotionEffectFrequency', 'PushEffect', 'PushEffectFrequency', 'Flip', 'FlipFrequency', 'SetNextPlayer', 'SetValue', 'SetValueFrequency', 'SetCount', 'SetCountFrequency', 'SetRotation', 'SetRotationFrequency', 'StepEffect', 'SlideEffect', 'LeapEffect', 'ByDieMove', 'MaxDistance', 'ReplacementCaptureFrequency', 'HopCaptureMoreThanOne', 'DirectionCapture', 'DirectionCaptureFrequency', 'EncloseCaptureFrequency', 'CustodialCapture', 'CustodialCaptureFrequency', 'InterveneCapture', 'InterveneCaptureFrequency', 'SurroundCapture', 'SurroundCaptureFrequency', 'CaptureSequence', 'CaptureSequenceFrequency', 'Group', 'Loop', 'Pattern', 'PathExtent', 'Territory', 'Fill', 'CanNotMove', 'Threat', 'CountPiecesMoverComparison', 'ProgressCheck', 'RotationalDirection', 'SameLayerDirection', 'ForwardDirection', 'BackwardDirection', 'BackwardsDirection', 'LeftwardDirection', 'RightwardsDirection', 'LeftwardsDirection', 'ForwardLeftDirection', 'ForwardRightDirection', 'BackwardLeftDirection', 'BackwardRightDirection', 'SameDirection', 'OppositeDirection', 'NumPlayPhase', 'LineLoss', 'LineLossFrequency', 'LineDraw', 'ConnectionEnd', 'ConnectionEndFrequency', 'ConnectionWinFrequency', 'ConnectionLoss', 'ConnectionLossFrequency', 'GroupEnd', 'GroupEndFrequency', 'GroupWin', 'GroupWinFrequency', 'GroupLoss', 'GroupDraw', 'LoopEnd', 'LoopEndFrequency', 'LoopWin', 'LoopWinFrequency', 'LoopLoss', 'PatternEnd', 'PatternEndFrequency', 'PatternWin', 'PatternWinFrequency', 'PathExtentEnd', 'PathExtentWin', 'PathExtentLoss', 'TerritoryEnd', 'TerritoryWin', 'TerritoryWinFrequency', 'Checkmate', 'CheckmateWin', 'NoTargetPieceEndFrequency', 'NoTargetPieceWin', 'NoTargetPieceWinFrequency', 'EliminatePiecesLoss', 'EliminatePiecesLossFrequency', 'EliminatePiecesDraw', 'EliminatePiecesDrawFrequency', 'NoOwnPiecesEnd', 'NoOwnPiecesWin', 'NoOwnPiecesLoss', 'NoOwnPiecesLossFrequency', 'FillEnd', 'FillEndFrequency', 'FillWin', 'FillWinFrequency', 'ReachWin', 'ReachLoss', 'ReachLossFrequency', 'ReachDraw', 'ReachDrawFrequency', 'ScoringLoss', 'ScoringLossFrequency', 'ScoringDraw', 'NoMovesLoss', 'NoMovesDrawFrequency', 'NoProgressEnd', 'NoProgressEndFrequency', 'NoProgressDraw', 'NoProgressDrawFrequency', 'BoardCoverageFull', 'BoardSitesOccupiedChangeNumTimes', 'BranchingFactorChangeLineBestFit', 'BranchingFactorChangeNumTimesn', 'DecisionFactorChangeNumTimes', 'MoveDistanceChangeSign', 'MoveDistanceChangeLineBestFit', 'PieceNumberChangeNumTimes', 'PieceNumberMaxIncrease', 'ScoreDifferenceMedian', 'ScoreDifferenceVariance', 'ScoreDifferenceChangeAverage', 'ScoreDifferenceChangeSign', 'ScoreDifferenceChangeLineBestFit', 'Math', 'Division', 'Modulo', 'Absolute', 'Exponentiation', 'Minimum', 'Maximum', 'Even', 'Odd', 'Visual', 'GraphStyle', 'MancalaStyle', 'PenAndPaperStyle', 'ShibumiStyle', 'BackgammonStyle', 'JanggiStyle', 'XiangqiStyle', 'ShogiStyle', 'TableStyle', 'SurakartaStyle', 'NoBoard', 'ChessComponent', 'KingComponent', 'QueenComponent', 'KnightComponent', 'RookComponent', 'BishopComponent', 'PawnComponent', 'FairyChessComponent', 'PloyComponent', 'ShogiComponent', 'XiangqiComponent', 'StrategoComponent', 'JanggiComponent', 'TaflComponent', 'StackType', 'Stack', 'ShowPieceValue', 'ShowPieceState', 'Implementation', 'StateType', 'StackState', 'VisitedSites', 'InternalCounter', 'SetInternalCounter', 'Efficiency','KintsBoard', 'FortyStonesWithFourGapsBoard', 'Roll', 'SumDice', 'CheckmateFrequency']
        
        df.drop(['Id',#Id has no practical meaning, it is just the line number.
         #The column with nunique=1 in train
         'Properties', 'Format', 'Time', 'Discrete', 'Realtime', 'Turns', 'Alternating', 'Simultaneous', 'HiddenInformation', 'Match', 'AsymmetricRules', 'AsymmetricPlayRules', 'AsymmetricEndRules', 'AsymmetricSetup', 'Players', 'NumPlayers', 'Simulation', 'Solitaire', 'TwoPlayer', 'Multiplayer', 'Coalition', 'Puzzle', 'DeductionPuzzle', 'PlanningPuzzle', 'Equipment', 'Container', 'Board', 'PrismShape', 'ParallelogramShape', 'RectanglePyramidalShape', 'TargetShape', 'BrickTiling', 'CelticTiling', 'QuadHexTiling', 'Hints', 'PlayableSites', 'Component', 'DiceD3', 'BiasedDice', 'Card', 'Domino', 'Rules', 'SituationalTurnKo', 'SituationalSuperko', 'InitialAmount', 'InitialPot', 'Play', 'BetDecision', 'BetDecisionFrequency', 'VoteDecisionFrequency', 'ChooseTrumpSuitDecision', 'ChooseTrumpSuitDecisionFrequency', 'LeapDecisionToFriend', 'LeapDecisionToFriendFrequency', 'HopDecisionEnemyToFriend', 'HopDecisionEnemyToFriendFrequency', 'HopDecisionFriendToFriend', 'FromToDecisionWithinBoard', 'FromToDecisionBetweenContainers', 'BetEffect', 'BetEffectFrequency', 'VoteEffectFrequency', 'SwapPlayersEffectFrequency', 'TakeControl', 'TakeControlFrequency', 'PassEffectFrequency', 'SetCost', 'SetCostFrequency', 'SetPhase', 'SetPhaseFrequency', 'SetTrumpSuit', 'SetTrumpSuitFrequency', 'StepEffectFrequency', 'SlideEffectFrequency', 'LeapEffectFrequency', 'HopEffectFrequency', 'FromToEffectFrequency', 'SwapPiecesEffect', 'SwapPiecesEffectFrequency', 'ShootEffect', 'ShootEffectFrequency', 'MaxCapture', 'OffDiagonalDirection', 'Information', 'HidePieceType', 'HidePieceOwner', 'HidePieceCount', 'HidePieceRotation', 'HidePieceValue', 'HidePieceState', 'InvisiblePiece', 'End', 'LineDrawFrequency', 'ConnectionDraw', 'ConnectionDrawFrequency', 'GroupLossFrequency', 'GroupDrawFrequency', 'LoopLossFrequency', 'LoopDraw', 'LoopDrawFrequency', 'PatternLoss', 'PatternLossFrequency', 'PatternDraw', 'PatternDrawFrequency', 'PathExtentEndFrequency', 'PathExtentWinFrequency', 'PathExtentLossFrequency', 'PathExtentDraw', 'PathExtentDrawFrequency', 'TerritoryLoss', 'TerritoryLossFrequency', 'TerritoryDraw', 'TerritoryDrawFrequency', 'CheckmateLoss', 'CheckmateLossFrequency', 'CheckmateDraw', 'CheckmateDrawFrequency', 'NoTargetPieceLoss', 'NoTargetPieceLossFrequency', 'NoTargetPieceDraw', 'NoTargetPieceDrawFrequency', 'NoOwnPiecesDraw', 'NoOwnPiecesDrawFrequency', 'FillLoss', 'FillLossFrequency', 'FillDraw', 'FillDrawFrequency', 'ScoringDrawFrequency', 'NoProgressWin', 'NoProgressWinFrequency', 'NoProgressLoss', 'NoProgressLossFrequency', 'SolvedEnd', 'Behaviour', 'StateRepetition', 'PositionalRepetition', 'SituationalRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation', 'Clarity', 'Narrowness', 'Variance', 'Decisiveness', 'DecisivenessMoves', 'DecisivenessThreshold', 'LeadChange', 'Stability', 'Drama', 'DramaAverage', 'DramaMedian', 'DramaMaximum', 'DramaMinimum', 'DramaVariance', 'DramaChangeAverage', 'DramaChangeSign', 'DramaChangeLineBestFit', 'DramaChangeNumTimes', 'DramaMaxIncrease', 'DramaMaxDecrease', 'MoveEvaluation', 'MoveEvaluationAverage', 'MoveEvaluationMedian', 'MoveEvaluationMaximum', 'MoveEvaluationMinimum', 'MoveEvaluationVariance', 'MoveEvaluationChangeAverage', 'MoveEvaluationChangeSign', 'MoveEvaluationChangeLineBestFit', 'MoveEvaluationChangeNumTimes', 'MoveEvaluationMaxIncrease', 'MoveEvaluationMaxDecrease', 'StateEvaluationDifference', 'StateEvaluationDifferenceAverage', 'StateEvaluationDifferenceMedian', 'StateEvaluationDifferenceMaximum', 'StateEvaluationDifferenceMinimum', 'StateEvaluationDifferenceVariance', 'StateEvaluationDifferenceChangeAverage', 'StateEvaluationDifferenceChangeSign', 'StateEvaluationDifferenceChangeLineBestFit', 'StateEvaluationDifferenceChangeNumTimes', 'StateEvaluationDifferenceMaxIncrease', 'StateEvaluationDifferenceMaxDecrease', 'BoardSitesOccupied', 'BoardSitesOccupiedMinimum', 'BranchingFactor', 'BranchingFactorMinimum', 'DecisionFactor', 'DecisionFactorMinimum', 'MoveDistance', 'MoveDistanceMinimum', 'PieceNumber', 'PieceNumberMinimum', 'ScoreDifference', 'ScoreDifferenceMinimum', 'ScoreDifferenceChangeNumTimes', 'Roots', 'Cosine', 'Sine', 'Tangent', 'Exponential', 'Logarithm', 'ExclusiveDisjunction', 'Float', 'HandComponent', 'SetHidden', 'SetInvisible', 'SetHiddenCount', 'SetHiddenRotation', 'SetHiddenState', 'SetHiddenValue', 'SetHiddenWhat', 'SetHiddenWho',
          #Columns that are present in the training set but not in the test set
         'num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1',
        'EnglishRules', 'LudRules','agent1','agent2',
         #object column - these are empty
         'Behaviour', 'StateRepetition', 'Duration', 'Complexity', 'BoardCoverage', 'GameOutcome', 'StateEvaluation', 'Clarity', 'Decisiveness', 'Drama', 'MoveEvaluation', 'StateEvaluationDifference', 'BoardSitesOccupied', 'BranchingFactor', 'DecisionFactor', 'MoveDistance', 'PieceNumber', 'ScoreDifference','selection1', 'selection2', 'exploration_const1', 'exploration_const2', 'playout1', 'playout2', 'score_bounds1', 'score_bounds2',
        ]+drop_cols,axis=1,inplace=True,errors='ignore')#Columns that are not in the test set can be ignored directly.
        
        df=self.reduce_mem_usage(df)
        print(f"feature_count:{len(df.columns)}")
        print("-"*30)
        return df

    
    def RMSE(self,y_true,y_pred):
        return np.sqrt(np.mean((y_true-y_pred)**2))
    class RmseObjective(object):
        def calc_ders_range(self, approxes, targets, weights):
            assert len(approxes) == len(targets)
            if weights is not None:
                assert len(weights) == len(approxes)

            result = []
            for index in range(len(targets)):
                # Gradient (first derivative)
                der1_1 = approxes[index] - targets[index] 
                der1_2 = -1
                if approxes[index] > targets[index]:
                    der1_2 = 1
                der1 = der1_1 + der1_2
                
                
                # Second derivative (set to 0 for simplicity in RMSE)
                der2 = 1

                if weights is not None:
                    der1 *= weights[index]
                    der2 *= weights[index]

                result.append((der1, der2))
            return result
    def train_model(self,):
#         self.train=self.FE(self.train,mode='train')
        #https://www.kaggle.com/code/ravi20076/mcts2024-mlmodels-v1/notebook
        
        taskType = "CPU"
        if torch.cuda.is_available():
            print("GPU is enabled.")
            print("GPU device name:", torch.cuda.get_device_name(0))
            taskType = "GPU"
        else:
            print("Using CPU")
      
            
        cat_model_1 = CatBoostRegressor(
#             task_type=taskType,
                eval_metric         = "RMSE",
               loss_function        = self.RmseObjective(),
               bagging_temperature  = 0.50,
               iterations           = 3096,
               learning_rate       = 0.08,
               max_depth          = 12,
               l2_leaf_reg         = 1.25,
               min_data_in_leaf    = 24,
               random_strength     = 0.25, 
               verbose             = 0)
        
        cat_model_2 = CatBoostRegressor(
#             task_type=taskType,
                eval_metric         = "RMSE",
               loss_function        = self.RmseObjective(),
               bagging_temperature  = 0.40,
               iterations           = 3096,
               learning_rate       = 0.08,
               max_depth          = 10,
               l2_leaf_reg         = 1.25,
               min_data_in_leaf    = 24,
               random_strength     = 0.20, 
                max_bin             = 2048,
               verbose             = 0)
       
        

       
        models=[
#                 (cat_model_1,'cat1'),
                (cat_model_2,'cat2'),
               ]
        
        for (model,model_name) in models:
            print("start training")
            X=self.train.drop([self.target,'GameRulesetName'],axis=1)
            GameRulesetName=self.train['GameRulesetName']
            y=self.train[self.target]
            oof_preds=np.zeros(len(X))
            
            y_int=round(y*15)

            def weight(y):
                if y in [-1,0,1]:
                    return 1
                else:
                    return 2.5
            
            sgkf = StratifiedGroupKFold(n_splits=self.num_folds,random_state=2024,shuffle=True)

            for fold, (train_index, valid_index) in (enumerate(sgkf.split(X,y_int,GameRulesetName))):
                print(f"fold:{fold}")

                X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
                y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

                sample_weight=y_train.apply(lambda x:weight(x))
                
                if 'lgb' in model_name:
                    model.fit(X_train,y_train,eval_set=[(X_valid, y_valid)],
                                 sample_weight=sample_weight,
                                 callbacks=[log_evaluation(500),early_stopping(100)]) 
                    #1000个迭代器,1%的特征
                    importances=model.feature_importances_
                    columns=X.columns
                    drop_cols=[]
                    for i in range(len(columns)):
                        if importances[i]<10:
                            drop_cols.append(columns[i])
                    print(f"drop_cols={drop_cols}")

    
                elif 'cat' in model_name:
                    model.fit(X_train, y_train,eval_set=(X_valid, y_valid), early_stopping_rounds=100, verbose=0)
                else:
                    model.fit(X_train,y_train,eval_set=[(X_valid, y_valid)],verbose=500)

                oof_preds[valid_index] = np.clip( model.predict(X_valid),-1.000,1.000)
               
                
                print(f"RMSE :{self.RMSE(y[valid_index].values,oof_preds[valid_index])}")

                self.pickle_dump(model,f'{model_name}_{fold}.model')
                self.model_paths.append((model_name,fold))

                del X_train,X_valid,y_train,y_valid
                gc.collect()
  
            print("For model ",model_name)
            print(f"RMSE :{self.RMSE(y.values,oof_preds)}")
    
            df = pd.DataFrame({'Utility':y.values, "Predicted":oof_preds, "Sq_Error":np.abs(y.values-oof_preds)})
            df.to_csv("Predictions.csv")
            
            
    def infer_model(self,test):
        test=self.FE(test,mode='test')
        test.drop(['GameRulesetName'],axis=1,inplace=True)
        test_preds=[]
        for i in range(len(self.model_paths)):
            model_name,fold=self.model_paths[i]
            test_copy=self.CV_feats(test.copy(),mode='test',model_name=model_name,fold=fold)
            model=self.pickle_load(f'{model_name}_{fold}.model')
#             test_preds+=[np.clip(model.predict(test_copy),-1.000,1.000)]
            test_preds+=[np.clip(model.predict(test_copy)*1.15,-0.985,0.985)]
        print(test_preds)
        return np.mean(test_preds,axis=0)

In [39]:
train_=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
train_=train_.to_pandas()

# train = train_.sample(n=1000, random_state=42)
train = train_
#print(f"len(train):{len(train)}")
test=pl.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv")
test=test.to_pandas()
print(f"len(test):{len(test)}")
#test.head()

len(test):3


In [40]:
preprocessor=Preprocessor(num_folds=5,train=train)
preprocessor.train = preprocessor.FE(preprocessor.train,mode='train')

preprocessor.train_model()  

FE:train
deal with outliers
agent1 agent2 feature
one_hot_encoder
Memory usage of dataframe is 590.10 MB
Memory usage after optimization is: 166.15 MB
Decreased by 71.8%
feature_count:350
------------------------------
Using CPU
start training
fold:0
RMSE :0.4673182923415952
fold:1
RMSE :0.4654463937459206
fold:2
RMSE :0.4020388745860109
fold:3
RMSE :0.4454648213354342
fold:4
RMSE :0.44699673738701884
For model  cat2
RMSE :0.4458751404203082


In [1]:
df = pd.read_csv("/kaggle/working/Predictions.csv")
def RMSE(y_true,y_pred):
        return np.sqrt(np.mean((y_true-y_pred)**2))
print(RMSE(df['Utility'].values,np.clip(df['Predicted'].values,-1.000,1.000)))

NameError: name 'pd' is not defined